In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys

# Make inline plots in this notebook, make them reasonable sizes, and with legible font size
%matplotlib inline
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = [12, 8]

# Note:  your sys.path needs to have the path to the jbolo/python functions defined.
# For example, on my mac running anaconda python, I insert it in
# ~/.ipython/profile_default/ipython_config.py
#  ## lines of code to run at IPython startup.
#c.InteractiveShellApp.exec_lines = [
#    "sys.path.append('/Users/ruhl/code/jbolo/python')"
#    ]
# Or, you can do this in every notebook you use this stuff in:
# sys.path.insert(0,'/Users/ruhl/code/jbolo/python/')

In [3]:
# List what yaml files you have so you can choose one for the next line.
!ls yamls 

CHLAT_20211210.yaml          SO_S4_comparison
SAT_HF_20211210.yaml         SPLAT_20211210.yaml
SAT_HF_20220216.yaml         SPLAT_20211210_bandfile.yaml
SAT_LFMF_20211210.yaml       spt3g.yaml
SAT_LFMF_20220216.yaml       test.yaml


In [4]:

%run python/RunOnce.py yamls/CHLAT_20211210.yaml

# or, if you want an output pickle file to store the whole sim:
# %run python/jbolo.py yamls/SPLAT_20211210_bandfile.yaml --pkl_out='output_pickle_filename.pkl'

# Note that you can run jbolo.py from the command line if you prefer,
# python python/jbolo.py yamls/SPLAT_20211210_bandfile.yaml

20211210 : CHLAT
                      LF_1       LF_2       MF_1       MF_2       HF_1       HF_2
 det_bandcenter :    25.700     38.740     91.450    148.450    226.990    285.500  
  det_bandwidth :     8.400     17.500     28.900     40.900     58.100     59.100  
 sys_bandcenter :    26.090     39.655     92.657    149.462    227.505    285.581  
  sys_bandwidth :     6.543     13.424     23.087     35.890     55.686     58.828  
 sky_bandcenter :    26.072     41.814     92.899    152.633    228.983    278.379  
  sky_bandwidth :     6.537     14.190     23.135     36.665     56.084     57.437  
          P_opt :     0.267      1.368      1.549      4.701     12.450     16.905  
          P_sat :     0.750      4.180      4.600     12.510     31.700     42.430  
         F_link :     0.524      0.524      0.524      0.524      0.524      0.524  
      G_dynamic :    21.040    117.265    129.048    350.954    889.308   1190.327  
           dpdt :     0.015      0.059      0.054  

In [5]:
# Print any on channel's optical elements table, basic properties
import jbolo_funcs as jbf
for ch in sim['channels'].keys():
    print('-----------------')
    jbf.print_optics(sim,ch)

-----------------
LF_1
Element            Popt(pW)   Effic  Effic_cumul
detector       :    0.0000     0.6300    1.0000
objlens        :    0.0000     0.9931    0.6300
lyot           :    0.0277     0.2521    0.6256
lowpass        :    0.0001     0.9400    0.1577
colimlens      :    0.0001     0.9931    0.1482
fieldlens      :    0.0001     0.9931    0.1472
nylon_filt     :    0.0000     0.9897    0.1462
alumina_filt   :    0.0042     0.9750    0.1447
RTMLI_10       :    0.0097     0.9960    0.1411
window         :    0.0053     0.9888    0.1405
secondary      :    0.1079     0.9754    0.1389
primary        :    0.0196     0.9954    0.1355
atmosphere     :    0.0593     0.9842    0.1349
cmb            :    0.0331     0.0000    0.1328
P_optical_total =  2.6698e-13
-----------------
LF_2
Element            Popt(pW)   Effic  Effic_cumul
detector       :    0.0000     0.6500    1.0000
objlens        :    0.0001     0.9926    0.6500
lyot           :    0.0288     0.4800    0.6452
lowpass   

In [8]:
%run python/RunOnce.py yamls/SPLAT_20211210.yaml

20211210 : SPLAT
                       ULF       LF_1       LF_2       MF_1       MF_2       HF_1       HF_2
 det_bandcenter :    19.950     25.700     38.740     91.450    148.450    226.990    285.500  
  det_bandwidth :     4.900      8.400     17.500     28.900     40.900     58.100     59.100  
 sys_bandcenter :    20.121     26.090     39.655     92.657    149.460    227.501    285.575  
  sys_bandwidth :     4.043      6.543     13.425     23.090     35.899     55.713     58.850  
 sky_bandcenter :    20.191     26.175     42.971     92.950    150.121    227.862    282.156  
  sky_bandwidth :     4.058      6.566     14.604     23.143     36.048     55.835     58.193  
          P_opt :     0.132      0.225      1.419      1.527      3.745      9.030     11.810  
          P_sat :     0.400      0.750      4.180      4.600     12.510     31.700     42.430  
         F_link :     0.524      0.524      0.524      0.524      0.524      0.524      0.524  
      G_dynamic :    11.22

In [10]:
# Print any on channel's optical elements table, basic properties
import jbolo_funcs as jbf
for ch in sim['channels'].keys():
    print('-----------------')
    jbf.print_optics(sim,ch)

-----------------
ULF
Element            Popt(pW)   Effic  Effic_cumul
detector       :    0.0000     0.6500    1.0000
objlens        :    0.0000     0.9933    0.6500
lyot           :    0.0193     0.2592    0.6456
lowpass        :    0.0001     0.9400    0.1674
colimlens      :    0.0000     0.9933    0.1573
fieldlens      :    0.0000     0.9933    0.1563
nylon_filt     :    0.0000     0.9899    0.1552
alumina_filt   :    0.0020     0.9761    0.1537
RTMLI_10       :    0.0061     0.9960    0.1500
window         :    0.0023     0.9891    0.1494
tertiary       :    0.0511     0.9795    0.1478
secondary      :    0.0012     0.9995    0.1447
primary        :    0.0012     0.9995    0.1447
atmosphere     :    0.0267     0.9882    0.1446
cmb            :    0.0220     0.0000    0.1428
P_optical_total =  1.3210e-13
-----------------
LF_1
Element            Popt(pW)   Effic  Effic_cumul
detector       :    0.0000     0.6300    1.0000
objlens        :    0.0000     0.9931    0.6300
lyot       

In [9]:
# Print any on channel's optical elements table, basic properties
import jbolo_funcs as jbf
for ch in sim['channels'].keys():
    print('-----------------')
    jbf.print_detector(sim,ch)

-----------------
ULF
  P_sat:          0.40
  F_link:         0.52
  G_dynamic:   1.12e-11
  NEP_phonon:     2.88
  NET_uncorr:   339.10
  NET_corr:     411.68
  Corr_factor:   1.214
  NET_NC_wafer:     51.59
  NET_C_wafer:     62.63
-----------------
LF_1
  P_sat:          0.75
  F_link:         0.52
  G_dynamic:   2.10e-11
  NEP_phonon:     3.95
  NET_uncorr:   291.24
  NET_corr:     356.15
  Corr_factor:   1.223
  NET_NC_wafer:     33.23
  NET_C_wafer:     40.64
-----------------
LF_2
  P_sat:          4.18
  F_link:         0.52
  G_dynamic:   1.17e-10
  NEP_phonon:     9.32
  NET_uncorr:   270.52
  NET_corr:     282.91
  Corr_factor:   1.046
  NET_NC_wafer:     30.87
  NET_C_wafer:     32.28
-----------------
MF_1
  P_sat:          4.60
  F_link:         0.52
  G_dynamic:   1.29e-10
  NEP_phonon:     9.78
  NET_uncorr:   291.49
  NET_corr:     334.90
  Corr_factor:   1.149
  NET_NC_wafer:     11.09
  NET_C_wafer:     12.74
-----------------
MF_2
  P_sat:         12.51
  F_link:  